In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


In [2]:
def preprocess_data(csv_file, sequence_length):
    
  
    df = pd.read_csv(csv_file)
    
    features = df[['magnitude', 'cdi', 'mmi', 'dmin', 'gap', 'depth', 'latitude', 'longitude']].values
    

    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    
    sequences = []
    for i in range(0, len(normalized_features) - sequence_length + 1, sequence_length // 2):  
        sequence = normalized_features[i:i + sequence_length]
        sequences.append(sequence)
    
    return torch.FloatTensor(sequences), scaler

In [3]:
normal_sequences, normal_scaler = preprocess_data("Normal.csv", 50)
anomalous_sequences, anomalous_scaler = preprocess_data("Anomalous.csv", 50)


/tmp/ipykernel_47305/52243508.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400268359/work/torch/csrc/utils/tensor_new.cpp:261.)
  return torch.FloatTensor(sequences), scaler


In [4]:

from TsunoGan import EnhancedAnoGAN, TsunamiDetector


In [5]:
# Initialize the model
sequence_length = 50  # Sequence length for training (you already have it)
latent_dim = 100  # Latent dimension for the generator (can be adjusted)
lambda_ano = 0.1  # Weight for anomaly loss
num_features = 8  # Number of features (as in the input data)


# Create the TsunamiDetector instance
tsunami_detector = TsunamiDetector(sequence_length, latent_dim, lambda_ano, num_features,device='cpu')

# Train the model on the normal sequences (normal_sequences is already preprocessed)
batch_size = 32
epochs = 100

# Convert normal_sequences to a DataLoader compatible format (TensorDataset)
normal_data = TensorDataset(normal_sequences)
normal_dataloader = DataLoader(normal_data, batch_size=batch_size, shuffle=True)

# Train the model
tsunami_detector.train(normal_dataloader, epochs=epochs)



Epoch 0: G_loss: 0.7269, D_loss: 0.7009
Epoch 10: G_loss: 0.6994, D_loss: 0.6729
Epoch 20: G_loss: 0.8766, D_loss: 0.6027
Epoch 30: G_loss: 0.9920, D_loss: 0.4188
Epoch 40: G_loss: 1.2661, D_loss: 0.4137
Epoch 50: G_loss: 1.5360, D_loss: 0.2694
Epoch 60: G_loss: 1.3942, D_loss: 0.3263
Epoch 70: G_loss: 2.0144, D_loss: 0.1804
Epoch 80: G_loss: 2.0841, D_loss: 0.1615
Epoch 90: G_loss: 2.4833, D_loss: 0.1012


In [7]:
# Calibrate threshold with normal data
validation_data = normal_sequences  # You can use a validation set or a portion of normal data
threshold_percentile = 95  # You can adjust this percentile based on your needs
threshold = tsunami_detector.calibrate_threshold(validation_data, percentile=threshold_percentile)

print(f"Calibrated threshold: {threshold}")


Calibrated threshold: 2.6446691155433655


In [8]:
# Assuming anomalous_sequences is a tensor of anomalous data (like actual tsunami events)
anomalous_sequences = torch.FloatTensor(anomalous_sequences)  # Ensure it's a tensor if not already

# Evaluate anomalous data
for sequence in anomalous_sequences:
    result = tsunami_detector.detect_tsunami(sequence)
    print(f"Is Tsunami: {result['is_tsunami']}, Confidence: {result['confidence']}, Anomaly Score: {result['anomaly_score']}")


Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.8137266635894775
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 2.1949853897094727
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.949520468711853
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.6330344676971436
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.9007203578948975
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.8562805652618408
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.8144011497497559
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.5864073038101196
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.41500723361969
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.572488784790039
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.687887191772461
Is Tsunami: False, Confidence: 0.0, Anomaly Score: 1.8075538873672485
